In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import json
import subprocess
import os
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect
import pandas as pd
from netCDF4 import Dataset

# Issues
# PS is invalid

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')
cursor=conn.cursor()

In [4]:
q="""
insert into sci.merra2_l72_3hr
select 
try_cast(CLOUD as real) CLOUD,
try_cast(DELP as real) DELP,
try_cast(EPV as real) EPV,
try_cast(H as real) H,
try_cast(O3 as real) O3,
try_cast(OMEGA as real) OMEGA,
try_cast(PHIS as real) PHIS,
try_cast(PL as real) PL,
try_cast(PS as real) PS,
try_cast(QI as real) QI,
try_cast(QL as real) QL,
try_cast(QV as real) QV,
try_cast(RH as real) RH,
try_cast(SLP as real) SLP,
try_cast(T as real) T,
try_cast(U as real) U,
try_cast(V as real) V,
try_cast(lat as real) lat,
try_cast(level as real) level,
try_cast(lon as real) lon,
date_add('second', try_cast(try_cast(time  as double) as bigint), timestamp '1970-01-01 00:00:00') time,
try_cast(date_trunc('day', date_add('second', try_cast(try_cast(time  as real) as bigint), timestamp '1970-01-01 00:00:00')) as date) date_part
from 
 sci_staging.merra2_l72_3hr
order by
 try_cast(lon as real)

"""

In [5]:
inputs={}
f = open("M2I3NVASM.txt", "r")
#ShortName:M2I3NVASM   # Daily file, contains 3 hr intervals
x=f.read()
f.close()
print (inputs)
urls=x.split('\n')
for u in urls:
    if len(u) == 0:
        continue
    #print (u)
    fileName=(u[u.rindex('/')+1:])
    fparts = fileName.split('.')
    #print (fparts)
    sortkey = fparts[2]
    dskey   = fparts[1]
    if sortkey in inputs:
        inputs[sortkey][dskey]['url']=u
        inputs[sortkey][dskey]['fn']=fileName
    else:
        inputs[sortkey]= {
            "inst3_3d_asm_Nv" : { "url" : "", "fn" : ""},
          }
        inputs[sortkey][dskey]['url']=u
        inputs[sortkey][dskey]['fn']=fileName

print (json.dumps(inputs, indent=1))

{}
{
 "20210101": {
  "inst3_3d_asm_Nv": {
   "url": "https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NVASM.5.12.4/2021/01/MERRA2_400.inst3_3d_asm_Nv.20210101.nc4",
   "fn": "MERRA2_400.inst3_3d_asm_Nv.20210101.nc4"
  }
 },
 "20210102": {
  "inst3_3d_asm_Nv": {
   "url": "https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NVASM.5.12.4/2021/01/MERRA2_400.inst3_3d_asm_Nv.20210102.nc4",
   "fn": "MERRA2_400.inst3_3d_asm_Nv.20210102.nc4"
  }
 },
 "20210103": {
  "inst3_3d_asm_Nv": {
   "url": "https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NVASM.5.12.4/2021/01/MERRA2_400.inst3_3d_asm_Nv.20210103.nc4",
   "fn": "MERRA2_400.inst3_3d_asm_Nv.20210103.nc4"
  }
 },
 "20210104": {
  "inst3_3d_asm_Nv": {
   "url": "https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NVASM.5.12.4/2021/01/MERRA2_400.inst3_3d_asm_Nv.20210104.nc4",
   "fn": "MERRA2_400.inst3_3d_asm_Nv.20210104.nc4"
  }
 },
 "20210105": {
  "inst3_3d_asm_Nv": {
   "url": "https://goldsmr5.gesdisc.eosdis.na

In [14]:
def do_it(m_str, m, t, outs): # t = 1 of 8 time dimensions
    #print (m['tavgM_2d_adg_Nx'])
    vars={}
    for d in m:
        ds=m[d]['ds']
        for v in ds.variables:
            if v in vars:
                print ('duplicate variable: ', v, ' skipping ....')
            else:
                #print (v)
                ds.set_auto_mask(False)
                if '_FillValue' in ds[v].ncattrs():
                    tmp=ds[v][t][:]
                    tmp[tmp == ds[v]._FillValue] = np.nan
                    vars[v]=tmp[:]
                else:
                    if v == 'time' or v=='lat' or v=='lon' or v=='lev':
                        vars[v]=ds[v][:]
                    else:
                        vars[v]=ds[v][t][:]
        #print (d)
        
# Normalize lat/lon

    vm_lat, vm_lev, vm_lon = np.meshgrid( vars['lat'], vars['lev'], vars['lon'] )
    #print (vm_lat.shape, vm_lon)

# Normalize time

    v_time = nc.num2date(vars['time'][t],ds['time'].units, only_use_python_datetimes=True)
    print (v_time)
    vm_time=np.zeros(14971392, dtype='datetime64[s]')
    vm_time[:]= v_time
    print (vm_time[0])

    
# PS(time, lat, lon)

# Normalize 2d variables
    vn_PHIS=np.zeros((72,361,576))
    vn_PS=np.zeros((72,361,576))
    vn_SLP=np.zeros((72,361,576))

    for i in range(72):
        vn_PHIS[i]= vars['PHIS']
        vn_PS[i]  = vars['PS']
        vn_SLP[i] = vars['SLP']

    stack_this = []
    ctr=0
    for v in sorted(vars):
        #print (ctr, v, vars[v].shape)
        ctr+=1
        if v == 'time':
                vm_time[:] = vm_time
                #print ('vmtime', vm_time.shape)
                stack_this.append(vm_time.astype('float'))
        elif v == 'lon':
                stack_this.append(vm_lon.flatten()[:])
        elif v == 'lat':
                stack_this.append(vm_lat.flatten()[:])
        elif v == 'lev':
                stack_this.append(vm_lev.flatten()[:])
        elif v == 'PHIS':
                print (vn_PHIS.shape)
                stack_this.append(vn_PHIS.flatten()[:])
        elif v == 'PS':
                stack_this.append(vn_PS.flatten()[:])
        elif v == 'SLP':
                stack_this.append(vn_SLP.flatten()[:])
        else:
                #print (ctr, 'shape=', vars[v].shape)
                stack_this.append(vars[v].flatten()[:])
        #print (stack_this)
    x=np.dstack( (stack_this) )
    fname = '/home/ec2-user/merra2-3d_3hr_' + m_str + '_' + str(t) + '.tsv.gz'
#    np.savetxt(fname, x[0], fmt='%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.3f~%.1f~%.3f~%.1f', delimiter='~', newline="\n")
    np.savetxt(fname, x[0], fmt='%.12g', delimiter='~', newline="\n")
    outs.append(fname)
        
    return

In [ ]:
for m in inputs:
    if m <= "20210731":
        continue
    print (m)
    for d in inputs[m]:
        #print (m, d)
        print (inputs[m][d]['url'], inputs[m][d]['fn'])
        subprocess.run(["curl", "-n", "-c", ".urs_cookies", "-b", ".urs_cookies", "-LJO", "--url", inputs[m][d]['url']])
        inputs[m][d]['ds']=Dataset(inputs[m][d]['fn'])

    t_size=inputs[m][d]['ds']['time'].size
    outs=[]
    for t in range(t_size):
        print (t)
        do_it(m, inputs[m], t, outs) # Create 3 hr files for this day.

    for d in inputs[m]:
        inputs[m][d]['ds'].close()
        os.remove(inputs[m][d]['fn'])
        
    stagingS3  = 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/'
    for f in outs:
        rc=subprocess.run(["aws", "s3", "cp", f, stagingS3 ])
#    print ('rc:', rc)
    # Execute insert into ...
    cursor.execute(q)
    print(cursor.description)
    print(cursor.fetchall())
    
    for f in outs:
        
    # Delete s3 staging files
        rc=subprocess.run(["aws", "s3", "rm", 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/' + f.rsplit('/', 1)[-1]])
        print ('rc:',rc)
     
    # Delete tsv.gz file(s)
        os.remove(f)


20210801
https://goldsmr5.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I3NVASM.5.12.4/2021/08/MERRA2_401.inst3_3d_asm_Nv.20210801.nc4 MERRA2_401.inst3_3d_asm_Nv.20210801.nc4
0
2021-08-01 00:00:00
2021-08-01T00:00:00
(72, 361, 576)
1
2021-08-01 03:00:00
2021-08-01T03:00:00
(72, 361, 576)
2
2021-08-01 06:00:00
2021-08-01T06:00:00
(72, 361, 576)
3
2021-08-01 09:00:00
2021-08-01T09:00:00
(72, 361, 576)
4
2021-08-01 12:00:00
2021-08-01T12:00:00
(72, 361, 576)
5
2021-08-01 15:00:00
2021-08-01T15:00:00
(72, 361, 576)
6
2021-08-01 18:00:00
2021-08-01T18:00:00
(72, 361, 576)
7
2021-08-01 21:00:00
2021-08-01T21:00:00
(72, 361, 576)
[('rows', 'bigint', None, None, 19, 0, 'UNKNOWN')]
[]
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210801_0.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210801_1.tsv.gz'], returncode=0)
rc: CompletedProcess(ar

In [38]:
print (vars['PHIS'][1])
vn_PHIS=np.zeros((72,361,576))
vn_PHIS[1]=vars['PHIS'][1]
print ('*********************************************')
print (vn_PHIS[1].flatten()[:])
print (np.amin(vars['PHIS'][1]), np.amax(vars['PHIS'][1]))
print (np.amin(vn_PHIS[1].flatten()[:]), np.amax(vn_PHIS[1].flatten()[:]))

[[27577.709 27577.709 27577.709 ... 27577.709 27577.709 27577.709]
 [28305.709 28305.709 28297.709 ... 28329.709 28321.709 28313.709]
 [29361.709 29353.709 29337.709 ... 29393.709 29385.709 29377.709]
 ...
 [    0.        0.        0.    ...     0.        0.        0.   ]
 [    0.        0.        0.    ...     0.        0.        0.   ]
 [    0.        0.        0.    ...     0.        0.        0.   ]]
*********************************************
[27577.70898438 27577.70898438 27577.70898438 ...     0.
     0.             0.        ]
-508.79028 54313.71
-508.790283203125 54313.7109375


In [32]:
#for m in inputs['20210801']:
#    print (inputs['20210801'][m])
#    break
ds1=inputs['20210801']['inst3_3d_asm_Nv']['ds']
print(ds1)
time=1
lat=180
lon=300
lev=71
print (ds1['time'][time], ds1['lat'][lat], ds1['lon'][lon], ds1['lev'][lev],ds1['H'][time][lev][lat][lon], ds1['PS'][time][lat][lon])
vars={}
for v in ds1.variables:
    print (v)
    vars[v]=ds1[v]
for v in sorted(vars):
    #print (v, vars[v][:])
    print (v, np.amin(vars[v][:]), np.amax( vars[v][:] ) )

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Comment: GMAO filename: d5124_m2_jun21.inst3_3d_asm_Nv.20210801.nc4
    Filename: MERRA2_401.inst3_3d_asm_Nv.20210801.nc4
    Conventions: CF-1
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2I3NVASM
    GranuleID: MERRA2_401.inst3_3d_asm_Nv.20210801.nc4
    ProductionDateTime: Re-processed file generated: Sat Nov 27 13:22:33 2021 GMT
    LongName: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    Title: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    SouthernmostLatitude: -90.0
    NorthernmostLatitude: 90.0
    WesternmostL

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type


180 -1.7975103014118005e-13 7.5 72.0 66.42889 101463.58
lon
lat
lev
time
CLOUD
DELP
EPV
H
O3
OMEGA
PHIS
PL
PS
QI
QL
QV
RH
SLP
T
U
V
CLOUD 0.0 0.9999955
DELP 1.0000002 5137.8384
EPV -4.7006793 1.8150109
H 14.346853 78270.93
O3 6.738275e-09 3.8556864e-05
OMEGA -6.727418 8.35052
PHIS -508.79028 54313.71
PL 1.5000001 103154.64
PS 51633.71 103934.24
QI 0.0 0.0007004738
QL 0.0 0.00093126297
QV 7.327706e-20 0.029852975
RH 0.0 1.02
SLP 94661.35 103934.24
T 162.56062 319.31848
U -103.28219 193.35548
V -87.64757 80.184975
lat -90.0 90.0
lev 1.0 72.0
lon -180.0 179.375
time 0 1260


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type


In [10]:
cursor.execute(q)
print(cursor.description)
print(cursor.fetchall())

[('rows', 'bigint', None, None, 19, 0, 'UNKNOWN')]
[]


In [14]:
for f in outs:
    # Delete s3 staging files
    rc=subprocess.run(["aws", "s3", "rm", 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/' + f.rsplit('/', 1)[-1]])
    print ('rc:',rc)
     
    # Delete tsv.gz file(s)
    #os.remove(f)

rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_0.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_1.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_2.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_3.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_4.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_5.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr

In [3]:
ds2=Dataset("/home/ec2-user/MERRA2_400.inst3_3d_asm_Nv.20211231.nc4", "r")

In [5]:
print (ds2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    History: Original file generated: Wed Jan 12 00:59:19 2022 GMT
    Comment: GMAO filename: d5124_m2_jan10.inst3_3d_asm_Nv.20211231.nc4
    Filename: MERRA2_400.inst3_3d_asm_Nv.20211231.nc4
    Conventions: CF-1
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2I3NVASM
    GranuleID: MERRA2_400.inst3_3d_asm_Nv.20211231.nc4
    ProductionDateTime: Original file generated: Wed Jan 12 00:59:19 2022 GMT
    LongName: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    Title: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    Southernmost

In [5]:
v_time = nc.num2date(ds2['time'][:],ds2['time'].units, only_use_python_datetimes=True)
v_lev  = ds2['lev'][:]
v_lon  = ds2['lon'][:]
v_lat  = ds2['lat'][:]
v_CLOUD = np.around(ds2['CLOUD'][:],7)
v_DELP = np.around(ds2['DELP'][:],7 )
v_EPV = np.around(ds2['EPV'][:], 7)
v_H = np.around(ds2['H'][:], 7)
v_O3 = np.around(ds2['O3'][:], 7)
v_OMEGA = np.around(ds2['OMEGA'][:], 7)
v_PHIS = np.around(ds2['PHIS'][:], 7)
v_PL = np.around(ds2['PL'][:], 7)
v_PS = ds2['PS'][:]
v_QI = ds2['QI'][:]
v_QL = ds2['QL'][:]
v_QV = ds2['QV'][:]
v_RH = ds2['RH'][:]
v_SLP = np.around(ds2['SLP'][:],7)
v_T = ds2['T'][:]
v_U = ds2['U'][:]
v_V = ds2['V'][:]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  if __name__ == '__main__':


In [6]:
import datetime
print (v_time)
#print (v_time[0].replace())
#x=v_time[0].strftime("%Y-%m-%d %H:%M:%S")
#print (x, type(x))
#y=datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
#print (y, type(y))
#print (z, type(z))
s_time=[]
for t in v_time:
    s_time.append(t.strftime("%Y-%m-%d %H:%M:%S").replace( ' ', 'T'))

print (s_time)

[cftime.DatetimeGregorian(2021, 12, 31, 0, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 3, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 6, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 9, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 12, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 15, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 18, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 21, 0, 0, 0, has_year_zero=False)]
['2021-12-31T00:00:00', '2021-12-31T03:00:00', '2021-12-31T06:00:00', '2021-12-31T09:00:00', '2021-12-31T12:00:00', '2021-12-31T15:00:00', '2021-12-31T18:00:00', '2021-12-31T21:00:00']


In [7]:
#lon=[-180, -90, 0, 90, 180]
#lat=[-90, 0, 90]
#m_lat,m_lon = np.meshgrid(lat,lon)
#print (m_lon.shape, m_lon)
#print (m_lat.shape, m_lat)

In [8]:
#print ('1', np.repeat([[1,2],[3,4]], 2))
#x = np.array( [[1,2],[3,4]] )
#print ('2a', np.repeat(x, 2))
#[     C C
#R    [1,2],
#R    [3,4]
#]
#print ('2b', np.repeat(x, 5, axis=1))
#print ('2c', np.repeat(x, 5, axis=0))
#print ('3', np.repeat(x, 3, axis=1))
#print ('4', np.repeat(x, [1, 2], axis=0))
#y=np.zeros((5, 2, 2))
#print (x.shape, y.shape)
#for i in range(5):
#    y[i]=x
#print (y[0])

vn_PHIS=np.zeros((72,361,576))
vn_PS=np.zeros((72,361,576))
vn_SLP=np.zeros((72,361,576))

for i in range(72):
    vn_PHIS[i]=v_PHIS[0]
    vn_PS[i]=v_PS[0]
    vn_SLP[i]=v_SLP[0]

print (vn_PS.shape)



(72, 361, 576)


In [9]:
charar = np.chararray(100, itemsize=19)
charar[:] = s_time[0]
#print (charar)
d = np.zeros(10, dtype='datetime64[s]')
#print (d)
d[:]=v_time[0]
print (d)

['2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00']


In [10]:
#vf_CLOUD = v_CLOUD.flatten()
#vf_H     = v_H.flatten()
#print (v_SLP.shape)
#X, Y = np.meshgrid(x*geo_var.perspective_point_height, y*geo_var.perspective_point_height)
#vm_lon, vm_lat = np.meshgrid(v_lon, v_lat)
print (vn_PHIS.shape, v_PL.shape)


vm_time=np.zeros(14971392, dtype='datetime64[s]')

vm_lat, vm_lev, vm_lon = np.meshgrid(v_lat, v_lev, v_lon)


t_size=v_time.size

for t in range(t_size):
#    print (vn_SLP[t].shape)
#    print (t, v_time[t])
#    print (vm_lat.shape, vm_lon.shape)
#    print (vn_PS.shape, vn_PS[t].shape)
    
    vm_time[:] = v_time[t]
    print (vm_time.shape)
    x=np.dstack((
        vm_lon.flatten(), 
        vm_lat.flatten(), 
        vm_lev.flatten(),
        vm_time.astype('float'),
        v_CLOUD[t].flatten(), 
        v_DELP[t].flatten(),
        v_EPV[t].flatten(),
        v_H[t].flatten(),
        v_O3[t].flatten(),
        v_OMEGA[t].flatten(),
        vn_PHIS.flatten(), 
        v_PL[t].flatten(), 
        vn_PS.flatten(), 
        v_QI[t].flatten(), 
        v_QL[t].flatten(), 
        v_QV[t].flatten(), 
        v_RH[t].flatten(), 
        vn_SLP.flatten(), 
        v_T[t].flatten(), 
        v_U[t].flatten(), 
        v_V[t].flatten()
    ))
    fname = '/home/ec2-user/test' + s_time[t].replace(':', '_') + '.tsv.gz'
    np.savetxt(fname, x[0], fmt='%.5f', delimiter='~', newline="\n")


(72, 361, 576) (8, 72, 361, 576)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)


In [26]:
np.savetxt('test.tsv.gz', x[0], fmt='%.5f', delimiter='~', newline="\n")

In [37]:
print (x.shape)
for i in range(x.size):
    v_x=x[0][i]
    print (i, v_x)
    #np.savetxt('test.tsv', v_x, fmt='%.5f', delimeter='~', newline="\n")
    break
print (i, x.shape)
print (x[0][0].min())


(1, 14971392, 8)
0 [-1.80000000e+02 -9.00000000e+01  1.00000000e+00  2.75777090e+04
  6.86709297e+04  9.97232578e+04  0.00000000e+00  1.00000024e+00]
0 (1, 14971392, 8)
-180.0


In [16]:
#for l in range(v_lev.size):
#    print (l, v_lev[l])
    
lon=1
lat=140
level=1
print (v_T[0][level][lat][lon])

205.59442


In [6]:
v_time = nc.num2date(ds2['time'][:],ds2['time'].units, only_use_python_datetimes=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  if __name__ == '__main__':
